In [0]:
df_csv = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/Volumes/workspace/default/training_volume/sample_orders.csv")

df_csv.show()
df_csv.createOrReplaceTempView("orders")

In [0]:
%sql
SELECT * FROM orders

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, rand

spark = SparkSession.builder.getOrCreate()

df = spark.range(0, 1500)

df = df.withColumn("order_id", col("id")) \
       .withColumn("country",
           when(col("id") % 3 == 0, "India")
           .when(col("id") % 3 == 1, "USA")
           .otherwise("Germany")
       ) \
       .withColumn("amount", (rand() * 1000).cast("int")) \
       .drop("id")

df.show(5)
df.count()



In [0]:
df.write \
  .mode("overwrite") \
  .partitionBy("country") \
  .parquet("/Volumes/workspace/default/training_volume/partitioned_parquet_orders")

In [0]:
df = spark.read.parquet(
    "/Volumes/workspace/default/training_volume/partitioned_parquet_orders"
)

df.show(5)
df.printSchema()



In [0]:
df_csv.write \
    .mode("overwrite") \
    .parquet("/Volumes/workspace/default/training_volume/orders_parquet")


In [0]:
df_parquet = spark.read.parquet(
    "/Volumes/workspace/default/training_volume/orders_parquet"
)

df_parquet.count()
df_parquet.show(5)

In [0]:
csv_files = dbutils.fs.ls("/Volumes/workspace/default/training_volume/sample_orders.csv")
csv_files
csv_size = sum(f.size for f in csv_files)
csv_size

parquet_files = dbutils.fs.ls("/Volumes/workspace/default/training_volume/orders_parquet")
parquet_files
parquet_size = sum(f.size for f in parquet_files)
parquet_size

print(f"CSV size: {csv_size / (1024*1024):.2f} MB")
print(f"Parquet size: {parquet_size / (1024*1024):.2f} MB")



In [0]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/Volumes/workspace/default/training_volume/sample_orders.csv")

df.show(5)
df.printSchema()

df.write \
    .format("delta") \
    .mode("overwrite") \
    .save("/Volumes/workspace/default/training_volume/orders_delta")

df_delta = spark.read \
    .format("delta") \
    .load("/Volumes/workspace/default/training_volume/orders_delta")

df_delta.show(5)
df_delta.count()


In [0]:
%sql

